<a href="https://colab.research.google.com/github/vn322/Synergy_DBM_301_CS/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9C%D0%B0%D1%80%D0%BA_%D0%B3%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Марк — это генеративная языковая модель, которая умеет придумывать новостные статьи
https://tproger.ru/articles/testim-marka-kak-proishodit-generaciya-novostej/
языковые модели — это когда мы пытаемся найти закономерности и правильные речевые последовательности в человеческом языке, а потом на основе этого пробуем создать “язык” независимо от самого человека.


Можно оценить вероятность появления какого-то словосочетания в языке. Иными словами, каждый раз мы отвечаем себе на вопрос: корректно ли использовать такую последовательность слов (формулировку) в русском языке? Если это допустимо, то вероятность употребления такого словосочетания достаточно велика, а значит мы используем языковую конструкцию, которую могут понять остальные, знающие русский.

В противном случае появляются конструкции вроде “лето наступила”, по правилам нашего языка составленные некорректно, а значит, вероятность появления такой конструкции в речи довольно мала.

Главная идея: правильные языковые конструкции могут быть созданы не только человеком, но и искусственным интеллектом (или роботом). Соответственно между хорошей языковой моделью (которая хорошо понимает правила нашего языка) и человеком можно наладить продуктивную взаимную коммуникацию. И вот это уже идея на миллион!

В обработке естественного языка (NLP) за последние пару лет произошла настоящая революция, и Трансформеры играли в ней основную роль. Здесь есть о чем рассказать.

Трансформеры – модели для обработки естественного языка, запущенные в 2017-м году, известные, в основном, повышением эффективности обработки и понимания последовательных данных вроде перевода и анализа текста.

В отличие от Рекуррентных Нейронных Сетей (RNN), обрабатывающих входные данные с начала до конца, Трансформеры обрабатывают входные данные параллельно, и, следовательно, существенно сокращают сложность вычислений.
https://arxiv.org/abs/1810.04805 

https://habr.com/ru/post/487358/

#Импорт библиотек

In [ ]:
import pandas as pd

In [ ]:
!pip install transformers==4.19.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.4 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import StoppingCriteria, StoppingCriteriaList
import torch

In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else None

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Генерация

In [ ]:
path = 'AnyaSchen/news_gpt-3'
tokenizer = GPT2Tokenizer.from_pretrained(path)
model = GPT2LMHeadModel.from_pretrained(path).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/983 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
SPECIAL_TOKENS = {'bos_token':'<bos>','eos_token' :'<eos>', 'pad_token':'<pad>', 'sep_token': '<sep>'}
tokenizer.add_special_tokens(SPECIAL_TOKENS)

3

In [ ]:
class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords_ids:list):
      self.keywords = keywords_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
      if input_ids[0][-1] in self.keywords:
        print(input_ids)
        return True
      return False

In [ ]:
stop_criteria = KeywordsStoppingCriteria(tokenizer.encode(tokenizer.eos_token, return_tensors="pt").to(DEVICE))

## Задай свое начало предложения:



Мы задаём начало заголовка нашей новости в переменную input. Если вдруг нам нечего дать в начало, и мы нажимаем Enter, то подставляется специальный символ, обозначающий для нашей модели начало предложения

In [ ]:
inp = input('Введите тему: ')

Введите тему: Проблемы развития компьютерного спорта


берём наш входной текст и говорим (дословно): токенайзер, тебе нужно заэнкодить наше начало.

In [ ]:
inp = inp if len(inp) > 0 else tokenizer.bos_token #токен начала предложения
input_ids = tokenizer.encode(inp, return_tensors="pt").to(DEVICE)

берем  кусочек заголовка, пилим его на слова (это делает токенайзер) и преобразуем каждое слово в большие наборы чисел (это и делает энкодер), к которым можно получить доступ по уникальным, назначенным им номерам.

Зачем так?

Цель: пытаемся научить компьютер “понимать”  язык.

Компьютер хранит  буковки числами, например, в виде ASCII-кода, где каждому символу присваивается уникальный номер (1, 2, 3…), а потом оно конвертируется в нолики и единички. Так вот слова в такой системе просто набор чисел, например:

котик → 1000011101010000111110100010000101000011100010000111010.

Но в  языке у каждого слова есть смысловой окрас и значение: для нас грусть и радость имеют противоположные смыслы, а в целом — это чувства.
Компьютер должен ощущать эту разницу между семантикой слов, чтобы хорошо “говорить” на  языке, но для него это всё равно набор ноликов и единичек, не имеющих смысла. 

In [ ]:
input_ids

tensor([[35241, 23282,  3281,  7173,   434,  7928]], device='cuda:0')

- разбиваем предложения на кусочки, из которых могут быть составлены родственные слова (именно это и делает токенайзер в нашей модели)
- превращаем каждый кусочек в вектор из чисел, который будет отвечать за семантику.
- Каждому кусочку присваивается уникальный номер, и энкодер  возвращает именно его. Почему так? Чтобы общаться,  используем слова в определенной последовательности и не сообщаем собеседнику все трактовки каждого сказанного  слова, надеясь, что  понимание слов идентично. Так и с компьютером: наборы уникальных номеров — это его слова.

In [ ]:
with torch.no_grad():
      out = model.generate(input_ids,
                          do_sample=True,
                          num_beams=3,
                          temperature=2.0,
                          top_p=0.9,
                          max_length = 600,
                          stopping_criteria=StoppingCriteriaList([stop_criteria]),
                          eos_token_id=tokenizer.eos_token_id,
                          bos_token_id=tokenizer.bos_token_id,
                          ).to(DEVICE)
print(tokenizer.batch_decode(out, skip_special_tokens=False)[0])

Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.


Проблемы развития компьютерного спорта в России объяснили тем, что у игроков слабый мозг <sep> Российские ученые исследовали влияние игровой деятельности на когнитивные процессы детей младшего возраста, и пришли к выводу, что это воздействие не характерно только для взрослых. Результаты исследования представлены в журнале PLOS Biology. <eos>
